# Examples of inputs

And target pr for some samples

In [ ]:
%reload_ext autoreload

%autoreload 2

%reload_ext dotenv
%dotenv

import functools
import math
import string

import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from mlde_utils import cp_model_rotated_pole, dataset_split_path
from mlde_analysis import plot_map
from mlde_analysis.examples import em_timestamps
from mlde_analysis.data import open_dataset_split

In [ ]:
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
dataset_name="bham64_ccpm-4x_12em_psl-sphum4th-temp4th-vort4th_pr"
split="train"
percentiles={
    "DJF wet": {"season": "DJF", "percentile": 0.8, "variable": "pr"},
    "JJA wet": {"season": "JJA", "percentile": 0.8, "variable": "pr"},
}
ensemble_members = ["01", "04"]

In [ ]:
DS = open_dataset_split(dataset_name, split, ensemble_members)
DS

In [ ]:
variables = [
    "target_pr",
    "psl",
    "vorticity250", "vorticity850",
    "temp250", "temp850",
    "spechum250", "spechum850",
]

grid_spec = [
    ["vorticity250", "temp250", "spechum250", "psl"],
    ["vorticity850", "temp850", "spechum850", "target_pr"],
]

em_ts = em_timestamps(DS, percentiles=percentiles, overrides={})

em_ts

In [ ]:
for example_label, example_defn in em_ts.items():
    example_ds = DS.sel(ensemble_member=example_defn[0], time=example_defn[1])
      
    fig = plt.figure(layout='constrained', figsize=(5.5, 3))
    axd = fig.subplot_mosaic(grid_spec, subplot_kw={"projection": cp_model_rotated_pole},)
    
    for var in variables:
        ax = axd[var]

        if var.startswith("temp"):
            style = "temp"
            pressure = var[-3:]
            if pressure == "250":
                title = f"Temp.\n@ {pressure}hPa"
            else:
                title = f"@ {pressure}hPa"
        elif var.startswith("vorticity"):
            style = "vorticity"
            pressure = var[-3:]
            if pressure == "250":
                title = f"Vorticity\n@ {pressure}hPa"
            else:
                title = f"@ {pressure}hPa"
        elif var.startswith("spechum"):
            style = None
            pressure = var[-3:]
            if pressure == "250":
                title = f"Humidity\n@ {pressure}hPa"
            else:
                title = f"@ {pressure}hPa"
        elif var == "target_pr":
            style = "pr"
            title = "Hi-res\nprecip."
        elif var == "psl":
            style = None
            title = "Pressure\n@ sea-level"
        else:
            style = None
            
        example_da = example_ds[var]
        plot_map(
            example_da,
            ax,
            style=style,
            title=title,
            # add_colorbar=True,
        )
    print(f"{example_defn[1]} em{example_defn[0]}")
    plt.show()